In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display
import re
import requests
from datetime import datetime, timedelta

In [4]:
def get_entertainment_soup() :
    chrome_driver_path = '/Users/got/Documents/retune/t+im/chromedriver-mac/chromedriver'
    chrome_driver_service = webdriver.chrome.service.Service(chrome_driver_path)

    driver = webdriver.Chrome(service=chrome_driver_service)
    driver.maximize_window()
    driver.get("https://www.ticketmelon.com")

    wait = WebDriverWait(driver, 10) #timeout=10 sec

    scroll_distance = 1200
    scroll_script = f"window.scrollBy(0, {scroll_distance});"
    driver.execute_script(scroll_script)

    print("scroll to entertainment :" , scroll_distance, "px")

    entertainment_button = wait.until(
        EC.element_to_be_clickable(
            (By.CSS_SELECTOR, '.slick-slide.slick-active.slick-current')))
    entertainment_button.click()

    scroll_distance = 1500
    scroll_script = f"window.scrollBy(0, {scroll_distance});"
    driver.execute_script(scroll_script)

    print("scroll to see more :" , scroll_distance, "px")

    see_more_button = driver.find_element(
        By.CSS_SELECTOR, '.HomeStyled__EventSeeMore-sc-125xn4a-2.eMhTTB')
    driver.execute_script("arguments[0].click();", see_more_button)

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    driver.quit()

    return soup

# create initial df 

In [7]:
soup = get_entertainment_soup()
print(soup.prettify())

scroll to entertainment : 1200 px
scroll to see more : 1500 px
<html data-react-helmet="lang" lang="EN" style="height:100%">
 <head>
  <meta content="#3BBBE6" name="theme-color"/>
  <style>
   body { margin: 0 } /* custom! */
  </style>
  <style>
   body {
  font-size: 14px !important;
  color: #4a4a4a;
  -webkit-font-smoothing: antialiased;
  -moz-osx-font-smoothing: grayscale;
  -webkit-overflow-scrolling: touch;
}
body:lang(th) {
  font-family: 'Prompt';
}
*:focus {
  outline: 0 !important;
}
.nav .bar {
  height: 64px;
}
a {
  cursor: pointer;
}
.pages {
  margin-top: 64px;
}
.static-page-header {
  background-image: url(https://s3-ap-southeast-1.amazonaws.com/tm-public-file/header-helpcenter.png);
  background-size: 100% 200px;
  background-position: top center;
  background-repeat: no-repeat;
  height: 200px;
  margin-top: 0;
}
@media (max-width: 767px) {
  .pages {
    margin-top: 55px;
  }
  .static-page-header {
    background-size: 150% 200px;
  }
}
@media only screen and (mi

In [8]:
def create_event_list(soup):
    elements = soup.find_all('div',
                            class_='MelonEventCardStyled__EventCardContainer-sc-8kc3ju-0'
                            )  #event-date, event-name, event-location
    event_data = []
    for element in elements:

        event_info = {
            "title": element.find('p', class_='event-name').text,
            "date": element.find('p', class_='event-date').text,
            "location": element.find('p', class_='event-location').text,
            "url": "https://www.ticketmelon.com" + element.find('a').get('href')
        }
        event_data.append(event_info)
    return event_data

In [9]:
event_data = create_event_list(soup)
df = pd.DataFrame(event_data)
display(df)

,title,date,location,url
0,PELUPO 2024,1 Mar - 2 Mar,"The Fields, Siam Country Club, Pattaya",https://www.ticketmelon.com/pelupo/2024
1,Enchanted Flora Park,1 Nov - 29 Feb,Flora Park,https://www.ticketmelon.com/florapark/enchante...
2,"""Bake Me Please พิชิตใจนายสายหวาน” Special EP....",28 Jan,Major Ratchayothin Theatre 13,https://www.ticketmelon.com/rosestudio/bake-me...
3,UnKonscious Festival 2024,30 Jan - 4 Feb,Alexa Beach club,https://www.ticketmelon.com/unkonscious/unk24
4,Mystic Valley 2024,2 Feb - 4 Feb,Mountain Creek Golf Resort,https://www.ticketmelon.com/mysticvalley/mysti...
...,...,...,...,...
126,Splash Pool Party,13 Apr - 16 Apr,สวนน้ำ The Mall Korat,https://www.ticketmelon.com/acreative/splashpo...
127,เฟี้ยวติวัล บุรีรัมย์,27 Apr,สนามช้างอารีน่า,https://www.ticketmelon.com/fiewtival/fiewtiva...
128,PONG 47 ปี Rock Never Dies,27 Apr,IMPACT ARENA เมืองทองธานี,https://www.ticketmelon.com/classy-record/pong...
129,เฟี้ยวติวัล ขอนแก่น,11 May,ศูนย์ค้าส่งอู้ฟู่,https://www.ticketmelon.com/fiewtival/fiewtiva...


In [10]:
def extract_dates(date_str):
    date_parts = re.findall(r'(\d{1,2}\s*[A-Za-z]{3})', date_str)
    
    if len(date_parts) == 2:
        start_date, end_date = date_parts
    elif len(date_parts) == 1:
        start_date = end_date = date_parts[0]
    else:
        start_date = end_date = None
    
    return start_date.strip(), end_date.strip() 

def calculate_year(date_str):
    # Convert month abbreviations to numeric values
    month_values = {
        'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4,
        'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8,
        'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
    }
    
    # Extract the month from the date
    month = date_str.split()[1]
    
    if month_values.get(month):
        if month_values[month] >= 8:
            return 2023
        else:
            return 2024
    else:
        return None  # Handle invalid month

    
df[['startdate', 'enddate']] = df['date'].apply(extract_dates).apply(pd.Series)

df['startyear'] = df['startdate'].apply(calculate_year)
df['endyear'] = df['enddate'].apply(calculate_year)
df['startyear'] = df['startyear'].astype(str)
df['endyear'] = df['endyear'].astype(str)

df['startdate'] = df['startdate'] + ' ' + df['startyear']
df['startdate'] = pd.to_datetime(df['startdate'], format='%d %b %Y', errors='coerce')
df['enddate'] = df['enddate'] + ' ' + df['endyear']
df['enddate'] = pd.to_datetime(df['enddate'], format='%d %b %Y', errors='coerce')

df.drop(columns=['startyear', 'endyear'], inplace=True)
df = df.sort_values(by=['startdate', 'enddate'])

df.drop(columns=['date'], inplace=True)

In [11]:
df.drop_duplicates()

,title,location,url,startdate,enddate
5,Monet and Friends Alive Bangkok,"Attraction Hall, 6th Floor, ICONSIAM",https://www.ticketmelon.com/liveimpact-event/m...,2023-09-22,2024-02-14
1,Enchanted Flora Park,Flora Park,https://www.ticketmelon.com/florapark/enchante...,2023-11-01,2024-02-29
7,The Backyard 168: Back to the 90's Campfire Party,"The Backyard 168, Khao Yai",https://www.ticketmelon.com/thebackyard168/the...,2023-12-01,2024-02-25
8,NuNew 2nd Single Nu Meow Official Goods,Photobook Delivery,https://www.ticketmelon.com/mandeework/nunew2n...,2023-12-12,2024-01-12
9,"LOOOK AT YOU, LOOOK AT BANYAKART",BANYAKART Khaoyai,https://www.ticketmelon.com/banyakart/lookatba...,2023-12-21,2024-02-29
...,...,...,...,...,...
126,Splash Pool Party,สวนน้ำ The Mall Korat,https://www.ticketmelon.com/acreative/splashpo...,2024-04-13,2024-04-16
127,เฟี้ยวติวัล บุรีรัมย์,สนามช้างอารีน่า,https://www.ticketmelon.com/fiewtival/fiewtiva...,2024-04-27,2024-04-27
128,PONG 47 ปี Rock Never Dies,IMPACT ARENA เมืองทองธานี,https://www.ticketmelon.com/classy-record/pong...,2024-04-27,2024-04-27
129,เฟี้ยวติวัล ขอนแก่น,ศูนย์ค้าส่งอู้ฟู่,https://www.ticketmelon.com/fiewtival/fiewtiva...,2024-05-11,2024-05-11


# add zone, price, solddate

In [14]:
def getSoup(url) :
    chrome_driver_path = '/Users/got/Documents/retune/t+im/chromedriver-mac/chromedriver'
    chrome_driver_service = webdriver.chrome.service.Service(chrome_driver_path)
    driver = webdriver.Chrome(service=chrome_driver_service)
    driver.get(url)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    driver.quit()
    return soup

def getInfo(soup):
    types = []
    prices = []
    qtys = []

    elements = soup.find_all('div', class_='ticket--type')
    for element in elements :
        div_text = element.contents[0].strip()
        types.append(div_text)

    elements = soup.find_all('div', class_='ticket--price')
    for element in elements :
        prices.append(element.text)

    elements = soup.find_all('div', class_='ticket--qty')
    
    for element in elements :
        qtys.append(element.text)
    return types, prices, qtys

def createTypelist(title, types, prices, qtys):
    type_list = []
    for i in range(len(types)) :
        if prices[0].strip(" ") != 'Price' : continue #skip event that's not concert
        if (types[i] == 'Ticket Type') or (prices[i].strip(" ") =='Price') : continue #prevent header 

        #if SOLDOUT --> get today, if sale ended --> sale ended, else none
        solddate = None
        if qtys[i] == 'Sales Ended' : solddate = 'Sales Ended'
        elif qtys[i] == 'Sold Out' : solddate = datetime.now().strftime("%Y-%m-%d")

        type_info = {
            'title': title,
            'type': types[i],
            'price': prices[i].split(' ')[0],
            'solddate': solddate
        }
        type_list.append(type_info)

    return type_list

#(['Ticket Type', '2 DEC: GOODMOOD', '3 DEC: GOODMOOD', '4 DEC: GOODMOOD', '8 DEC: GOODMOOD', '9 DEC: GOODMOOD', '10 DEC: GOODMOOD', '11 DEC: GOODMOOD', '15 DEC: GOODMOOD', '16 DEC: GOODMOOD', '17 DEC: GOODMOOD', '[BUNDLE] 2 DEC: GOODMOOD & GOODFOOD', '[BUNDLE] 3 DEC: GOODMOOD & GOODFOOD', '[BUNDLE] 4 DEC: GOODMOOD & GOODFOOD', '[BUNDLE] 8 DEC: GOODMOOD & GOODHOOD', '[BUNDLE] 9 DEC: GOODMOOD & GOODHOOD', '[BUNDLE] 10 DEC: GOODMOOD & GOODHOOD', '[BUNDLE] 11 DEC: GOODMOOD & GOODHOOD', '[BUNDLE] 15 DEC: GOODMOOD & GOODHOOD', '[BUNDLE] 16 DEC: GOODMOOD & GOODHOOD', '[BUNDLE] 17 DEC: GOODMOOD & GOODHOOD'], 
# ['Price', '390 THB', '390 THB', '390 THB', '390 THB', '390 THB', '390 THB', '390 THB', '390 THB', '390 THB', '390 THB', '490 THB', '490 THB', '490 THB', '490 THB', '490 THB', '490 THB', '490 THB', '490 THB', '490 THB', '490 THB'], 
# ['Qty.', 'Sales Ended', 'Sales Ended', 'Sales Ended', 'Sales Ended', 'Sales Ended', 'Sales Ended', 'Sales Ended', ' 0 123456789101112131415', ' 0 123456789101112131415', ' 0 123456789101112131415', 'Sales Ended', 'Sales Ended', 'Sales Ended', 'Sales Ended', 'Sales Ended', 'Sales Ended', 'Sales Ended', ' 0 123456789101112131415', ' 0 123456789101112131415', ' 0 123456789101112131415'])

In [15]:
unique_concert = df['title'].unique()
today_date = datetime.now()
types_list = []
count =0
for i in range(len(unique_concert)) :
    count +=1
    title = unique_concert[i]
    starttime = time.time()
    print(title)
    enddate = df.loc[df['title']==title, 'enddate'].iloc[0] + timedelta(days=1) #add 1 days to include the enddate
    if enddate < today_date: continue
    url = df.loc[df['title']==title, 'url'].iloc[0]
    soup = getSoup(url)
    types, prices, qtys = getInfo(soup)
    endtime = time.time()
    print(endtime - starttime)
    if len(types) == 0 :         
        print('count =',count)
        count =0
        time.sleep(660)
        print('-------------------------------------------')

        starttime = time.time()
        print(title)
        soup = getSoup(url)
        types, prices, qtys = getInfo(soup)
        type_list = createTypelist(title, types, prices, qtys)
        types_list += type_list
        endtime = time.time()
        print(endtime - starttime)
        print('-------------------------------------------')
        continue
    print('-------------------------------------------')
    type_list = createTypelist(title, types, prices, qtys)
    types_list += type_list

temp = pd.DataFrame(types_list)
df = pd.merge(df, temp, on='title', how='outer')

Monet and Friends Alive Bangkok
7.329056978225708
-------------------------------------------
Enchanted Flora Park
11.861545085906982
-------------------------------------------
The Backyard 168: Back to the 90's Campfire Party
6.229838132858276
-------------------------------------------
NuNew 2nd Single Nu Meow Official Goods
6.2390642166137695
-------------------------------------------
LOOOK AT YOU, LOOOK AT BANYAKART
6.209290027618408
-------------------------------------------
Trevor Lock for One Month Only! Best Comedy on Khaosan
3.6766560077667236
-------------------------------------------
Trampolin 11 JAN 2024
5.943953275680542
-------------------------------------------
Space Odysses 11 JAN 2024
3.792654037475586
-------------------------------------------
Rock The Boat
6.324259996414185
-------------------------------------------
LIDO LIVEHOUSE presents 'YENTED & PLASUI PLASUI'
3.7816410064697266
-------------------------------------------
Blueprint Livehouse: Dept
3.654847

In [105]:
# df = pd.merge(df, temp, on='title', how='outer')

In [106]:
pd.set_option('display.max_row', None)

In [107]:
date = datetime.now().date()
print(date)
path = '/Users/got/Documents/retune/t+im/melon_solddate/melonSolddate'+'('+str(date)+')'+'.csv'
# print(path)
df.to_csv(path, index=False)

2023-12-29


# check solddate

- get old df
- scrape melon page for more concert
- check solddate

In [108]:
path = '/Users/got/Documents/retune/t+im/melon_solddate/melonSolddate(2023-12-13).csv'
df = pd.read_csv(path, header=None, skiprows=2)


In [109]:
df.columns = df.iloc[0]

In [110]:
df = df.drop(0)

In [111]:
df

,title,location,url,startdate,enddate,type,price,solddate
1,Monet and Friends Alive Bangkok,"Attraction Hall, 6th Floor, ICONSIAM",https://www.ticketmelon.com/liveimpact-event/m...,2023-09-22,2024-01-07,VIP Ticket: Come 2,"1,118",NaN
2,Monet and Friends Alive Bangkok,"Attraction Hall, 6th Floor, ICONSIAM",https://www.ticketmelon.com/liveimpact-event/m...,2023-09-22,2024-01-07,General Ticket: Come 2,743,NaN
3,Monet and Friends Alive Bangkok,"Attraction Hall, 6th Floor, ICONSIAM",https://www.ticketmelon.com/liveimpact-event/m...,2023-09-22,2024-01-07,Special Attention Ticket: Come 2,360,NaN
4,Monet and Friends Alive Bangkok,"Attraction Hall, 6th Floor, ICONSIAM",https://www.ticketmelon.com/liveimpact-event/m...,2023-09-22,2024-01-07,VIP Ticket: 5 for 4,954,NaN
5,Monet and Friends Alive Bangkok,"Attraction Hall, 6th Floor, ICONSIAM",https://www.ticketmelon.com/liveimpact-event/m...,2023-09-22,2024-01-07,General Ticket: 5 for 4,637,NaN
6,Monet and Friends Alive Bangkok,"Attraction Hall, 6th Floor, ICONSIAM",https://www.ticketmelon.com/liveimpact-event/m...,2023-09-22,2024-01-07,Special Attention Ticket: 5 for 4,307,NaN
7,Monet and Friends Alive Bangkok,"Attraction Hall, 6th Floor, ICONSIAM",https://www.ticketmelon.com/liveimpact-event/m...,2023-09-22,2024-01-07,VIP Ticket,"1,490",NaN
8,Monet and Friends Alive Bangkok,"Attraction Hall, 6th Floor, ICONSIAM",https://www.ticketmelon.com/liveimpact-event/m...,2023-09-22,2024-01-07,General Ticket,990,NaN
9,Monet and Friends Alive Bangkok,"Attraction Hall, 6th Floor, ICONSIAM",https://www.ticketmelon.com/liveimpact-event/m...,2023-09-22,2024-01-07,Special Attention Ticket,480,NaN
10,Monet and Friends Alive Bangkok,"Attraction Hall, 6th Floor, ICONSIAM",https://www.ticketmelon.com/liveimpact-event/m...,2023-09-22,2024-01-07,ACRYLIC PAINTING WORDSHOP,990,NaN


In [112]:
# soup = get_entertainment_soup()
# event_list = create_event_list(soup)
# print(event_list)